<a href="https://colab.research.google.com/github/JongHoB/Data_Mining_Spark/blob/main/Colab_2_Frequent_Pattern_Mining_in_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frequent Pattern Mining in Spark


**This** Colab, adopted for Ajou CSE3313, is from [Stanford CS246](http://cs246.stanford.edu).

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f0a5d277238ab959cd9bf24fb685f0daba822627d3266f94f29ab75216339081
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='12Aub5cpNgY0XGn-M_dUL8xp86NxLCUe2'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='12BkEQ1XyR_NOI_-oMyq9gEShFOAt9-cz'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [ ]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [ ]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [ ]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [ ]:
''' 2 lines of code expected '''
# YOUR CODE HERE
join=products.join(orders,products.product_id==orders.product_id)
table=join.groupBy("order_id").agg(collect_set("product_name").alias("product_name"))

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```



In [ ]:
''' 3 lines of code expected '''
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="product_name", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(table)

Compute how many frequent itemsets and association rules were generated by running FP-growth alongside visalizing top frequent itemsets and association rules.


In [ ]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
print(model.freqItemsets.count())
print(model.associationRules.count())
model.freqItemsets.sort(desc("freq")).show()
model.associationRules.sort(desc('confidence')).show()

120
0
+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.


In [ ]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="product_name", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(table)
print(model.freqItemsets.count())
print(model.associationRules.count())
model.freqItemsets.sort(desc("freq")).show()
model.associationRules.sort(desc('confidence')).show()

4444
11
+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+--------------------+--------------------+------------------+--

To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [ ]:
# YOUR CODE HERE
model.transform(table).show()

+--------+--------------------+--------------------+
|order_id|        product_name|          prediction|
+--------+--------------------+--------------------+
|       1|[Bag of Organic B...|                  []|
|      96|[Roasted Turkey, ...|                  []|
|     112|[Umcka Elderberry...|                  []|
|     218|[Okra, Black Plum...|                  []|
|     456|[Petite Peas, Lar...|                  []|
|     473|[Organic Whole Mi...|                  []|
|     631|[Organic Cilantro...|                  []|
|     762|[Organic Cucumber...|                  []|
|     774|[Nacho Cheese Sau...|                  []|
|     844|[Organic Red Radi...|                  []|
|     904|[Zero Calorie Col...|                  []|
|     988|[Whipped Light Cr...|                  []|
|    1032|[Organic Living B...|                  []|
|    1077|[Sparkling Water,...|                  []|
|    1119|[Shallot, Large L...|                  []|
|    1139|[Cinnamon Rolls w...|               